In [1]:
import requests, re, random, configparser, telegram, os
from bs4 import BeautifulSoup
from telegram.ext import Updater # 更新者
from telegram.ext import CommandHandler, CallbackQueryHandler # 註冊處理 一般用 回答用
from telegram.ext import MessageHandler, Filters # Filters過濾訊息
from telegram import InlineKeyboardMarkup, InlineKeyboardButton # 互動式按鈕
import logging, time
from flask import Flask, request, app
from imgurpython import ImgurClient
import re

In [2]:
# setting token
token = '6042405264:AAH8dHRC4hhXEoyIuEmXYp4D9YVN--zJPSI'
# initialize bot
updater = Updater(token=token, use_context=False)

/var/folders/f_/rnjcx52d45v_q1lj0ck1b9580000gn/T/ipykernel_5862/2953257587.py:4: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  updater = Updater(token=token, use_context=False)


In [3]:
# set a dispatcher
dispatcher = updater.dispatcher

In [4]:
# parser for exchange rate
target_url = 'https://rate.bot.com.tw/xrt/flcsv/0/day'
currency = {} # store every exchange rate in the dict
currency['NTD']=1 
rs = requests.session()
res = rs.get(target_url, verify=False)
res.encoding = 'utf-8'
rt = res.text
rts = rt.split('\n')
for i in rts:              # 讀取串列的每個項目
        try:                             # 使用 try 避開最後一行的空白行
            a = i.split(',')             # 每個項目用逗號拆分成子串列
            currency[a[0]]=float(a[12])         # 取出第一個 ( 0 ) 和第十三個項目 ( 12 )，存入字典
        except:
            continue

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rate.bot.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
# when typing in '/start'
def start (bot, update):
    message=update.message
    chat = message['chat']
    update.message.reply_text(text='HI  '+str(chat['first_name'])+str(chat['last_name']))# say hello to user
    update.message.reply_text(text='這是匯率機器人，使用範例：1000NTD=JPY')# introduce the bot
    update.message.reply_text(text='/help可瀏覽各貨幣縮寫，/rate可瀏覽今日牌告匯率')# introduce the commands that user can use

# when typing in '\help'
def help (bot, update):
    message=update.message
    chat = message['chat']
    # print out the tutorial message to users
    update.message.reply_text(text='使用範例：1000台幣換算成日幣，請輸入1000NTD=JPY')
    update.message.reply_text(text='各貨幣縮寫')
    update.message.reply_text(text='美金(USD) 港幣(HKD) 英鎊(GBP)')
    update.message.reply_text(text='澳幣(AUD) 日幣(JPY) 加拿大幣(CAD)')
    update.message.reply_text(text='紐元(NZD) 泰幣(THB) 新台幣(NTD)')
    update.message.reply_text(text='歐元(EUR) 韓元(KRW) 瑞士法郎(CHF)')
    update.message.reply_text(text='南非幣(ZAR) 瑞典幣(SEK) 菲國比索(PHP)')
    update.message.reply_text(text='印尼幣(IDR) 越南盾(VND) 馬來幣(MYR)')
    update.message.reply_text(text='人民幣(CNY) 新加坡幣(SGD)')

# when typing in '\rate'
def rate(bot, update):
    # parser for every exchange rate
    target_url = 'https://rate.bot.com.tw/xrt/flcsv/0/day'
    rs = requests.session()
    res = rs.get(target_url, verify=False)
    res.encoding = 'utf-8'
    rt = res.text
    rts = rt.split('\n')
    bot.sendMessage(chat_id=update.message.chat_id, text='今日貨幣對其他貨幣的牌告匯率:\n')
    for i in rts:              # 讀取串列的每個項目
        try:                             # 使用 try 避開最後一行的空白行
            a = i.split(',')             # 每個項目用逗號拆分成子串列
            bot.sendMessage(chat_id=update.message.chat_id, text=a[0] + ': ' + a[12])# 取出第一個 ( 0 ) 和第十三個項目 ( 12 )
        except:
            break

# when type in other commands, check if user want to change amount of the currency to another currency's.
def rateexchange(bot, update): # 其他訊息，換算
    message = update.message
    global currency

    #find numbers
    num = int(re.match('(\.?\d*)', message.text).group(0))
    #find words
    find_word = re.compile(r'[a-z]+',re.I)
    word = find_word.findall(message.text)
    
    # exchange calculation
    rate_1 = currency[f'{word[0].upper()}']
    rate_2 = currency[f'{word[1].upper()}']
    
    # their relative rate
    rate_now = rate_2/rate_1
    # do the calculation to turn certain amount of currency into another corresponding amount of another currency.
    money = num/rate_now
    # send message to users
    update.message.reply_text(f'匯率：{rate_now}')
    update.message.reply_text(f'{money:.2f} {word[1].upper()}')

    # if the user type in something that the bot cannot understand or find the exchange rate
    #update.message.reply_text('找不到對應匯率')

# based on different input messages, there are four different types of functions to do correspoding tasks
dispatcher.add_handler(CommandHandler('start',start))# /start
dispatcher.add_handler(CommandHandler('rate',rate))# /rate
dispatcher.add_handler(CommandHandler('help',help))# /help
dispatcher.add_handler(MessageHandler(Filters.text, rateexchange)) # Filters如果是文字就呼叫rateexchange


updater.start_polling()# start
updater.idle()# idling
updater.stop()# stop the bot